In [4]:
# MENGIMPOR SEMUA LIBRARY YANG DIPERLUKAN UNTUK BAB 5
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.svm import LinearSVC, SVC, LinearSVR, SVR

# Mengatur default plotting
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Untuk plot di dalam notebook (jika Anda menggunakan Jupyter)
# %matplotlib inline

print("--- BAB 5: SUPPORT VECTOR MACHINES (SVM) ---")
print("Semua library telah diimpor.")

# 1. KLASIFIKASI SVM LINEAR
print("\n[1. Klasifikasi SVM Linear]")
# Teori: Ide utama SVM adalah 'Large Margin Classification'.
# SVM tidak hanya mencari garis untuk memisahkan kelas, tetapi mencari
# "jalan" (margin) terluas di antara kelas.
#
# - Hard Margin: Semua data HARUS di luar jalan. Sensitif outlier.
# - Soft Margin: Keseimbangan antara jalan lebar & 'pelanggaran margin'
#   (data yang masuk ke jalan). Ini dikontrol oleh hyperparameter 'C'.
#   - C rendah: Jalan lebar, lebih banyak pelanggaran (regularisasi tinggi).
#   - C tinggi: Jalan sempit, lebih sedikit pelanggaran (regularisasi rendah).

# Menggunakan dataset Iris
iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # fitur: petal length, petal width
y = (iris["target"] == 2).astype(np.float64)  # target: 1 jika Iris virginica, 0 jika bukan

# PENTING: SVM SANGAT SENSITIF TERHADAP SKALA FITUR
# Kita harus menggunakan StandardScaler!
# Kita gunakan Pipeline untuk menggabungkan scaling dan model SVM.

# LinearSVC adalah implementasi SVM linear yang cepat.
# 'hinge' adalah loss function standar untuk SVM.
# max_iter disetel tinggi untuk memastikan konvergensi
svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1, loss="hinge", random_state=42, dual=True, max_iter=10000))
    ])

svm_clf.fit(X, y)

print("Model LinearSVC telah dilatih.")
print(f"Prediksi untuk [5.5, 1.7]: {svm_clf.predict([[5.5, 1.7]])}") # Akan memprediksi [1.]

# 2. KLASIFIKASI SVM NON-LINEAR
print("\n[2. Klasifikasi SVM Non-Linear]")
# Banyak dataset tidak dapat dipisahkan secara linear.

# Menggunakan dataset Moons
from sklearn.datasets import make_moons
X_moons, y_moons = make_moons(n_samples=100, noise=0.15, random_state=42)

# Fungsi helper untuk plot
def plot_dataset(X, y, axes):
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bs")
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "g^")
    plt.axis(axes)
    plt.grid(True, which='both')
    plt.xlabel(r"$x_1$", fontsize=20)
    plt.ylabel(r"$x_2$", fontsize=20, rotation=0)

# plot_dataset(X_moons, y_moons, [-1.5, 2.5, -1, 1.5])
# plt.title("Dataset Moons (Non-Linear)")
# plt.show() # Data ini tidak bisa dipisah garis lurus

# 2a. Metode 1: Fitur Polinomial
# Teori: Kita bisa menambahkan fitur baru (misal, x1^2, x2^2, x1*x2)
# untuk membuat data dapat dipisahkan secara linear di dimensi yang lebih tinggi.
print("  Melatih SVM Non-Linear dengan Fitur Polinomial...")

polynomial_svm_clf = Pipeline([
        ("poly_features", PolynomialFeatures(degree=3)),
        ("scaler", StandardScaler()),
        ("svm_clf", LinearSVC(C=10, loss="hinge", random_state=42, dual=True, max_iter=10000))
    ])

polynomial_svm_clf.fit(X_moons, y_moons)
print("  Model SVM dengan Fitur Polinomial (degree 3) telah dilatih.")

# 2b. Metode 2: Kernel Trick (Cara yang Jauh Lebih Efisien)
print("  Melatih SVM Non-Linear dengan Kernel Trick...")
# Teori: 'Kernel Trick' adalah trik matematika yang memungkinkan SVM
# mendapatkan hasil yang SAMA SEPERTI menggunakan fitur polinomial
# berderajat tinggi, tanpa benar-benar HARUS menambahkannya.
# Ini menghemat biaya komputasi secara drastis.

# Kernel Polinomial
poly_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
    ])
poly_kernel_svm_clf.fit(X_moons, y_moons)
print("  Model SVM dengan Kernel 'poly' telah dilatih.")

# Kernel Gaussian RBF (Radial Basis Function)
# Teori: Kernel RBF adalah kernel yang paling populer dan kuat.
# Ia bekerja dengan menambahkan fitur berdasarkan 'kedekatan'
# (similarity) setiap instance ke 'landmarks'.
#
# Hyperparameter utama:
# - 'C': Mengontrol soft margin (sama seperti LinearSVC).
# - 'gamma': Mengontrol 'lebar' kurva Gaussian.
#   - gamma kecil: Kurva lebar, decision boundary halus (regularisasi tinggi).
#   - gamma besar: Kurva sempit, boundary berliku (regularisasi rendah, bisa overfit).
rbf_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001)) # Contoh gamma besar, C kecil
    ])
rbf_kernel_svm_clf.fit(X_moons, y_moons)
print("  Model SVM dengan Kernel 'rbf' telah dilatih.")


# 3. REGRESI SVM (SVR)
print("\n[3. Regresi SVM (SVR)]")
# Teori: SVM juga bisa untuk regresi. Idenya dibalik:
# Alih-alih mencari "jalan" terluas DI ANTARA kelas,
# SVR mencari "jalan" (margin) yang bisa memuat SEBANYAK MUNGKIN
# instance DI DALAM jalan.
# Lebar jalan dikontrol oleh hyperparameter 'epsilon' (e).

# 3a. Linear SVR
print("  Melatih LinearSVR...")
# Membuat data linear palsu
np.random.seed(42)
m_reg = 50
X_reg = 2 * np.random.rand(m_reg, 1)
y_reg = (4 + 3 * X_reg + np.random.randn(m_reg, 1)).ravel() # .ravel() mengubah ke 1D

# 'epsilon' mengontrol lebar jalan (margin)
linear_svr = LinearSVR(epsilon=1.5, random_state=42, max_iter=10000)
linear_svr.fit(X_reg, y_reg)
print("  Model LinearSVR (epsilon=1.5) telah dilatih.")

# 3b. Nonlinear SVR (menggunakan Kernel Trick)
print("  Melatih SVR Non-Linear (kernel poly)...")
# Membuat data kuadratik palsu
m_reg_nl = 100
X_reg_nl = 2 * np.random.rand(m_reg_nl, 1) - 1
y_reg_nl = (0.2 + 0.1 * X_reg_nl + 0.5 * X_reg_nl**2 + np.random.randn(m_reg_nl, 1) / 10).ravel()

# Menggunakan SVR dengan kernel polinomial
svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X_reg_nl, y_reg_nl)
print("  Model SVR Non-Linear (degree=2) telah dilatih.")

# Plot hasil SVR Non-Linear (di-comment)
# X_new_nl = np.linspace(-1, 1, 100).reshape(100, 1)
# y_pred_nl = svm_poly_reg.predict(X_new_nl)
# plt.plot(X_reg_nl, y_reg_nl, "b.")
# plt.plot(X_new_nl, y_pred_nl, "r-", linewidth=2, label="Prediksi SVR")
# plt.xlabel("$x_1$")
# plt.ylabel("$y$")
# plt.legend()
# plt.title("SVR dengan Kernel Polinomial")
# plt.show()

print("\n--- Selesai Bab 5 ---")

--- BAB 5: SUPPORT VECTOR MACHINES (SVM) ---
Semua library telah diimpor.

[1. Klasifikasi SVM Linear]
Model LinearSVC telah dilatih.
Prediksi untuk [5.5, 1.7]: [1.]

[2. Klasifikasi SVM Non-Linear]
  Melatih SVM Non-Linear dengan Fitur Polinomial...
  Model SVM dengan Fitur Polinomial (degree 3) telah dilatih.
  Melatih SVM Non-Linear dengan Kernel Trick...
  Model SVM dengan Kernel 'poly' telah dilatih.
  Model SVM dengan Kernel 'rbf' telah dilatih.

[3. Regresi SVM (SVR)]
  Melatih LinearSVR...
  Model LinearSVR (epsilon=1.5) telah dilatih.
  Melatih SVR Non-Linear (kernel poly)...
  Model SVR Non-Linear (degree=2) telah dilatih.

--- Selesai Bab 5 ---
